In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
    Hyperrectangle,
    VPolytope
)

from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  

from util import (
    timeit,
    diditwork,
    INFO,
    YAY,
    WARN,
    ERROR,
)  # pylint: disable=import-error, no-name-in-module, unused-import

import plotly.graph_objects as go  # pylint: disable=import-error
from plotly.express.colors import sample_colorscale  # pylint: disable=import-error
import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [24]:
def plot_var(fig, mu, sigma, color="blue"):
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    theta = np.linspace(0, 2*np.pi, 300)
    ellipsis = (np.sqrt(eigenvalues[None,:]) * eigenvectors) @ [np.sin(theta), np.cos(theta)] + mu.reshape((2,1))
    fig.add_trace(go.Scatter(x=[mu[0]], y=[mu[1]], marker_color=color, showlegend=False))
    fig.add_trace(go.Scatter(x=ellipsis[0,:], y=ellipsis[1,:], marker_color=color, showlegend=True, name="ellipsoid"))

def get_clockwise_vertices(vpoly:VPolytope):
    vertices = list(vpoly.vertices().T)
    c = np.mean(vpoly.vertices(),axis=1) # compute center
    vertices.sort(key = lambda p: np.arctan2( (p-c)[1], (p-c)[0] ) )
    return np.array(vertices).T

def plot_hpoly(fig:go.Figure, hpoly):
    vpoly = VPolytope(hpoly)
    vertices = get_clockwise_vertices(vpoly)
    x = np.append(vertices[0,:], vertices[0,0])
    y = np.append(vertices[1,:], vertices[1,0])
    fig.add_trace(go.Scatter(x=x, y=y, mode = 'lines', showlegend=True, name="polyhedron", marker_color="red"))


In [27]:


hper = Hyperrectangle([0,0], [2,2])
hpoly = hper.MakeHPolyhedron() 
# hpoly = HPolyhedron(VPolytope( np.array([[0,0],[1,0],[2,1],[2,2],[1,2],[0,1]]).T ))
# hpoly = HPolyhedron(VPolytope( np.array([[0,0],[1,0],[2,1.5],[2,2],[1,2],[0,1]]).T ))
hpoly = HPolyhedron(VPolytope( np.array([[0,0],[2,0],[2,2],[1,2],[0,1]]).T ))


A, b = hpoly.A(), hpoly.b()
B = np.hstack( (b.reshape((len(b),1)), -A) )

one = 1
mu = hper.Center()
assert np.all(A.dot(mu) <= b*one)


prog = MathematicalProgram()
sigma = prog.NewSymmetricContinuousVariables(len(mu))
# sigma = prog.NewContinuousVariables(1)[0]*np.eye(2)

# M = prog.NewSymmetricContinuousVariables(len(mu)) + np.outer(mu,mu)
M = sigma + np.outer(mu,mu)
mat = np.vstack(( np.hstack((one, mu)), np.hstack((mu.reshape((len(mu),1)), M)) ))
prog.AddPositiveSemidefiniteConstraint(mat)

for e in B.dot(mat).dot(B.T).flatten():
    prog.AddLinearConstraint(e >= 0)
prog.AddMaximizeLogDeterminantCost(sigma)
solution = Solve(prog)
diditwork(solution)

sigma_sol = solution.GetSolution(sigma)

if isinstance(sigma_sol[0,0], Expression):
    funct = np.vectorize(lambda x: x.Evaluate())
    sigma_sol = funct(sigma_sol)
print(sigma_sol)


solve successful!
5.896002882492295e-09
SolutionResult.kSolutionFound
Solver is Mosek
time 0.0026030540466308594
rescode 0
solution_status 1
[[9.99999996e-01 7.42001699e-05]
 [7.42001699e-05 9.99999996e-01]]


In [28]:
fig = go.Figure()
plot_hpoly(fig, hpoly)
plot_var(fig, mu, sigma_sol)
fig.update_yaxes(range=[-1,3])   # Set y-axis limits
fig.update_xaxes(range=[-1,3])   # Set y-axis limits
fig.update_layout(height=600, width=700)

fig.update_layout(
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)
fig.show()

